# A PyNets Primer in Python and Bash

Docker/Singularity containers are the preferable way to run PyNets because the compute environment will include all optional dependencies, fully unlocked plotting capabilities, and will perform predictably and with fully reproducible numerical precision. To keep things simple for this demonstration, however, let's begin by just installing PyNets in a virtual environment and then run the workflow manually on some example data. The scope of this tutorial will cover single-subject workflows. For more examples (i.e. including usage with docker/singularity), see: https://pynets.readthedocs.io/en/latest/usage.html

Although we will explore the package interactively in the code that follows, bear in mind that PyNets is a *workflow*, not a library like its core dependencies (Dipy, Nilearn, Nipype, Networkx). Thus, demonstrating its usage is fundamentally a command-line endeavor, rather than a purely pythonic one to which you may be typically accustomed. 

PyNets was intentionally designed to scale with supercomputers, but also run using just a couple cores on your local laptop. Although this tutorial is designed to explore the latter use-case, ideally the user will also have (or acquire) **systems-level** thinking (an awareness of issues relevant to distributed/parallel computing) in order to make the most of what PyNets has to offer. Recall that the uniqueness of the PyNets workflow, unlike any other software for brain network modeling, is that it is intended to facilitate a more reproducible ensemble connectome learning which, to be performed efficiently, warrants the use of HPC or AWS cloud infrastructures. 

## Installation

In [1]:
%%bash
# Assuming that python3, pip, and FSL are already installed...
# Start a virtual environment and install some dependencies for our lesson.
pip install virtualenv --user
mkdir ~/virtualenvironment 2>/dev/null
virtualenv ~/virtualenvironment/pynets
cd ~/virtualenvironment/pynets/bin
source activate
./pip3 install -U gdown fury # for downloading data, running pynets, and some 3d viz
./pip3 install pynets=='0.9.999b'

Using base prefix '/usr/local/anaconda3'
New python executable in /Users/derekpisner/virtualenvironment/pynets/bin/python
Installing setuptools, pip, wheel...
done.
Requirement already up-to-date: gdown in /Users/derekpisner/virtualenvironment/pynets/lib/python3.7/site-packages (3.11.1)
Requirement already up-to-date: fury in /Users/derekpisner/virtualenvironment/pynets/lib/python3.7/site-packages (0.5.1)


ERROR: Could not find a version that satisfies the requirement pynets==0.9.999b (from versions: 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.3.0, 0.3.1, 0.3.2, 0.3.3, 0.3.5, 0.3.6, 0.3.7, 0.3.8, 0.3.9, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.4.8, 0.4.9, 0.4.70, 0.4.71, 0.4.72, 0.4.73, 0.4.74, 0.4.75, 0.4.77, 0.4.78, 0.4.80, 0.4.81, 0.4.82, 0.4.83, 0.4.84, 0.4.85, 0.4.86, 0.4.87, 0.4.88, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.5.4, 0.5.5, 0.5.6, 0.5.61, 0.5.62, 0.5.63, 0.5.64, 0.5.65, 0.5.66, 0.5.67, 0.5.68, 0.5.69, 0.5.70, 0.5.71, 0.5.72, 0.5.74, 0.5.75, 0.5.76, 0.5.77, 0.5.78, 0.5.79, 0.5.80, 0.5.81, 0.5.82, 0.5.83, 0.5.84, 0.5.85, 0.5.86, 0.5.88, 0.5.89, 0.5.90, 0.5.91, 0.5.92, 0.5.93, 0.5.94, 0.5.95, 0.5.96, 0.5.97, 0.5.98, 0.5.99, 0.6.1, 0.6.2, 0.6.3, 0.6.4, 0.6.5, 0.6.6, 0.6.10, 0.6.11, 0.6.12, 0.6.13, 0.6.14, 0.6.15, 0.6.17, 0.6.18, 0.6.19, 0.6.20, 0.6.21, 0.6.22, 0.6.23, 0.6.24, 0.6.25, 0.6.26, 0.6.27, 0.6.28, 0.6.29, 0.6.30, 0.6.31, 0.6

CalledProcessError: Command 'b"# Assuming that python3, pip, and FSL are already installed...\n# Start a virtual environment and install some dependencies for our lesson.\npip install virtualenv --user\nmkdir ~/virtualenvironment 2>/dev/null\nvirtualenv ~/virtualenvironment/pynets\ncd ~/virtualenvironment/pynets/bin\nsource activate\n./pip3 install -U gdown fury # for downloading data, running pynets, and some 3d viz\n./pip3 install pynets=='0.9.999b'\n"' returned non-zero exit status 1.

In [ ]:
%%bash
# The only GUI-based visualizer that I continue to use (personally) is fsleyes,
# which I find to be immensely intuitive, especially for fine-grained QC of overlays.
# But feel free to use an image viewer of your choice. Future PyNets versions will likely 
# include html-style reports.
# For macs, download this link for fsleyes:
wget https://fsl.fmrib.ox.ac.uk/fsldownloads/fsleyes/FSLeyes-latest-macos.tar.gz #for 2d viz
#rm -rf ~/virtualenvironment/FSLeyes.app
tar -xzvf FSLeyes-latest-macos.tar.gz -C ~/virtualenvironment
# For Linux, downnload the appropriate precompiled build from https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes

# Fetch sample preprocessed data

Now we can download a minimal dataset from OASIS that includes preprocessed, multimodal fMRI and dMRI data.

*Note*: Normally, we could just use a dataset from datalad or from s3 (which will download automatically using an s3:// file path prefix if your AWS credentials are properly configured!). See `pynets_cloud` CLI , with examples here: https://pynets.readthedocs.io/en/latest/usage.html#quickstart 

In [ ]:
%%bash
# Now we create an output directory for the derivatives of the pipeline (if one doesn't exist already).
if  [ -d ~/Downloads/.pynets ]; then
    rm -rf ~/Downloads/.pynets/test_oasis*
else
    mkdir ~/Downloads/.pynets
fi

# And download the data to a generic "derivatives" directory.
if  [ ! -f ~/Downloads/.pynets/test_oasis.tar.gz ]; then
    cd ~/Downloads/.pynets
    gdown https://drive.google.com/uc?id=1beEoc_Pdk6OBDYc80mBDTvUhcUny9Gu3 -O ~/Downloads/.pynets/test_oasis.tar.gz
else
    cd ~/Downloads/.pynets
fi

mkdir -p ~/Downloads/.pynets/derivatives/sub-OAS31172 2>/dev/null
tar -xzvf test_oasis.tar.gz -C derivatives/sub-OAS31172

Now we construct a command-line call for a single subject from the data we just downloaded. We can do this in two ways -- (1) using the `pynets_bids` API since our sample data is in BIDS format and can be queried using pybids; (2) with the `pynets` API for comparison.
So, for run 1 of session d0407 from subject OAS31172, lets sample an ensemble of 144 functional connectome estimates (1 models x 6 thresholds x 2 smoothing values x 2 high-pass filter thresholds x 3 atlases x 2 time-series extraction methods).

In [ ]:
%%bash
dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`

# BIDS way using a pre-configured .json file that specifies how we want the pipeline to run.
# We can view this file to get an idea of what it contains:
cat ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_bold.json

# Next we initiate the `pynets_bids` CLI (note the inclusion of a run_label since the BOLD acquisitions for this dataset contain two runs):%%bash
~/virtualenvironment/pynets/bin/pynets_bids "$abs_dir"/.pynets/derivatives "$abs_dir"/.pynets/outputs participant func --participant_label OAS31172 --session_label d0407 --run_label 1 -config ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_bold.json

# *Note that the configuration in `bids_config_bold.json` is equivalent to running the following (non-BIDS) CLI call that does not require a config file:
# ~/virtualenvironment/pynets/bin/pynets "$abs_dir"/.pynets/outputs -id OAS31172_d0407_1 -mod 'partcorr' -min_thr 0.20 -max_thr 0.80 -step_thr 0.10 -sm 0 4 -hp 0 0.028 -a 'BrainnetomeAtlasFan2016' 'atlas_harvard_oxford' 'destrieux2009_rois' -es 'mean' 'variance' -anat "$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/anat/sub-OAS31172_ses-d0407_run-01_T1w.nii.gz -func "$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/func/sub-OAS31172_ses-d0407_task-rest_run-01_bold.nii.gz -plug 'MultiProc' -work '/tmp/pynets_work' -mst -plt -embed

# Viewing outputs

In [ ]:
%%bash
cd ~/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/func
ls

tree

Lets do a bit of quality-control to ensure, for example, that the inverse warping of the destrieux2009_rois atlas from template-space to native T1w anatomical space is valid.

In [ ]:
%%bash
t1w_image=`ls /tmp/pynets_work/*/*/meta_wf_*/fmri_connectometry*/register_node/reg/imgs/*t1w_brain.nii.gz | head -1`
atlas_in_t1w_image=`ls /tmp/pynets_work/*_wf_single_subject_fmri*/wf_single_*/meta_wf_*/fmri_connectometry_*/_atlas_destrieux2009_rois/register_atlas_node/atlas_destrieux2009_rois/*_gm.nii.gz | head -1`

~/virtualenvironment/FSLeyes.app/Contents/*/fsleyes "$t1w_image" "$atlas_in_t1w_image" -cm 'random' &


In [ ]:
import glob
from pathlib import Path
from IPython.display import Image
Image(filename=glob.glob(str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/figures/*_glass_viz.png')[0])


Above is a glass brain depiction of bilateral regions of the Brainnetome atlas (Fan et al., 2016) using a partial correlation estimator, 4 fwhm smoothing, 0.028Hz high-pass filter, based on variance of the node-extracted time-series, with 20% post-hoc thresholding using the Minimum-Spanning Tree (MST) method. The latter method serves as an anti-fragmentation device that ensures we can prevent isolated (i.e. disconnected) nodes that can violate certain graph theoretical assumptions.

In the visualization, node size conveys the level of node importance (smaller is lower eigenvector centrality) and node color corresponds to hierarchical Louvain community affiliation (8 distinct communities found).

The below adjacency matrix depicts a single connectome estimate, with community affiliation. 

In [ ]:
Image(filename=glob.glob(str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/figures/*')[0])

We could also look at the mean connectome (i.e. across all 144 estimates) -- what we might from here on out refer to as an **omnetome** as well:

In [ ]:
%matplotlib inline
import glob
import numpy as np
from matplotlib import pyplot as plt
from nilearn.plotting import plot_matrix
from pynets.core.thresholding import standardize

mats = [np.load(i) for i in glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/func/BrainnetomeAtlasFan2016/graphs/*.npy')]

mean_mat = standardize(np.mean(mats, axis=0))

plot_matrix(
    mean_mat,
    figure=(10, 10),
    labels=[' ']*len(mean_mat),
    vmax=np.percentile(mean_mat[mean_mat > 0], 90),
    vmin=np.percentile(mean_mat[mean_mat > 0], 10),
    reorder="average",
    auto_fit=True,
    grid=False,
    colorbar=True,
    cmap='RdBu_r',
)
plt.show()

As you can see, we get a much more information-rich graph. This graph, unlike the first, now represents a new *distribution* of connectomes, that, by virtue of its plurality of views, more exhaustively samples from the true *population* of networks in this individual that may exhibit connectivity, across the whole brain as a region of interest, at any point in time during the course of the 5-10 minute resting-state time-series.

# Collecting Outputs
So, we explored the outputs of our connectome ensemble visually, but let's take a closer look at our omnetome's topology. To do this, we run another workflow using the `pynets_collect` CLI, which collects the various graph topological metrics extracted from each of the connectome point estimates in our ensemble.

In [ ]:
%%bash

dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`
~/virtualenvironment/pynets/bin/pynets_collect -basedir "$abs_dir"/.pynets/outputs -modality 'func'

In [ ]:
import pathlib
from pathlib import Path
import pandas as pd

# Now we can load a dataframe containing all the AUC topological graph metrics calculated for this particular subject's run:
p = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/func_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'
df_individual = pd.read_csv(p, index_col=False)
df_individual

# Note that if we were to sample connectomes from multiple subjects, the previous pynets_collect CLI would
# simply append new rows to the summary `all_subs_neat_func.csv` dataframe.
p = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/all_subs_neat_func.csv'
df_group = pd.read_csv(p, index_col=False)
df_group

The below multiplot depicts distributions of average graph topological metrics, calculated using Area-Under-the-Curve (AUC) across our window of multiple thresholds, for the ensemble of 144 connectomes sampled. As you can visually discern, topology varies considerably across estimates.

In [ ]:
%matplotlib inline
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from pynets.plotting.plot_gen import plot_graph_measure_hists

csv_all_metrics = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/func_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'

out = plot_graph_measure_hists(csv_all_metrics)
out

We could also plot the omnetome embeddings to visualize lower-dimensional latent positions of the ensemble corresponding to each distinct graph resolution

In [ ]:
%matplotlib inline
from graspy.plot import pairplot
import glob
import os
import numpy as np
from matplotlib import pyplot as plt

omnetomes = [[np.load(i), os.path.basename(i).split('.npy')[0]] for i in glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/func/*/embeddings/*omnetome.npy')]

for grad, title in omnetomes:
    plot = pairplot(grad, title=title + '_Functional')

# Structural Connectometry

And now we construct a command-line call for the same subject, but using their dMRI data instead. This time, we should ideally use a higher voxel resolution (1mm) since we are working with microstructure data, but this would increase runtime by as much as 50%, which would be better saved for HPC environments. For demonstration purposes, we will therefore downsample our data slightly and work in 2mm voxel resolution.

So, for run 1 of session d0407 from subject OAS31172, lets sample an ensemble of 72 structural connectome estimates (1 diffusion model type x 6 thresholds x 2 direction-getting methods x 2 minimum streamline length thresholds x 3 atlases.

In [ ]:
%%bash
%%capture

# Again, get the absolute paths to files and directories we will use.
# The CLI's in PyNets do NOT accept relative paths.
dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`

# BIDS way using a pre-configured .json file that specifies how we want the pipeline to run.
# We can view this file to get an idea of what it contains:
cat ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_dwi.json

# Next we initiate the `pynets_bids` CLI:
~/virtualenvironment/pynets/bin/pynets_bids "$abs_dir"/.pynets/derivatives "$abs_dir"/.pynets/outputs participant dwi --participant_label OAS31172 --session_label d0407 -config ~/virtualenvironment/pynets/lib/python3.7/site-packages/pynets/config/bids_config_dwi.json

# *Note that the configuration in `bids_config_dwi.json` is equivalent to running the following (non-BIDS) CLI call that does not require a config file:
#~/virtualenvironment/pynets/bin/pynets "$abs_dir"/.pynets/outputs -mod 'csa' -min_thr 0.20 -max_thr 0.80 -step_thr 0.10 -dg 'det' 'prob' -ml 20 0 -a 'BrainnetomeAtlasFan2016' 'atlas_harvard_oxford' 'destrieux2009_rois' -anat ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/anat/sub-OAS31172_ses-d0407_run-01_T1w.nii.gz" -dwi ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.nii.gz" -bval ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.bval" -bvec ""$abs_dir"/.pynets/derivatives/sub-OAS31172/ses-d0407/dwi/sub-OAS31172_ses-d0407_dwi.bvec" -id OAS31172_d0407_1 -plug 'MultiProc' -work '/tmp/pynets_work' -mst -plt -vox '2mm' -embed


# Viewing outputs

In [ ]:
%%bash
cd ~/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi
ls .
tree

Lets do a bit of quality-control to ensure, for example, that the inverse warping of the harvard_oxford atlas from template-space to native DWI anatomical space is valid.

In [ ]:
%%bash
t1w_dwi_image=`ls /tmp/pynets_work/*/*/meta_wf_*/dmri_connectometry*/register_node/dmri_reg/reg/imgs/t1w_in_dwi.nii.gz | head -1`
atlas_in_t1w_dwi_image=`ls /tmp/pynets_work/*/*/meta_wf_*/dmri_connectometry*/*/register_atlas_node/*/*atlas_dwi_track.nii.gz | head -1`
density_map=`ls ~/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/tractography/*.nii.gz | head -1`

~/virtualenvironment/FSLeyes.app/Contents/*/fsleyes "$t1w_dwi_image" "$atlas_in_t1w_dwi_image" -cm 'random' "$density_map" &

In [ ]:
import glob
from IPython.display import Image
Image(filename=glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/figures/OAS31172_d0407_modality-dwi_est-csa_nodetype-parc_samples-50000streams_tt-local_dg-det_ml-20_thr-0.2_glass_viz.png')[0])

Above is a glass brain depiction of regions of the Harvard-Oxford atlas using a tensor model estimator of diffusion, deterministic tractography, a minimum fiber length threshold of 20, with 80% post-hoc thresholding using the Minimum-Spanning Tree (MST) method.
Again, un the visualization, node size conveys the level of node importance (smaller is lower eigenvector centrality) and node color corresponds to hierarchical Louvain community affiliation (only two distinct communities found). Unlike in the functional case, however, edges are here depicted with dotted white lines to differentiate them from functional edges, which carry a different meaning.

In [ ]:
Image(filename=glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi/atlas_harvard_oxford/figures/OAS31172_d0407_modality-dwi_est-csa_nodetype-parc_samples-50000streams_tt-local_dg-det_ml-0_thr-0.8_adj_mat.png')[0])


The above adjacency matrix depicts a single connectome estimate, with community affiliation. But we could also look at a structural omnetome (i.e. based on FA-weighted fiber counts) across all 72 independent connectome estimations. Note that by default pynets only samples 10,000 streamlines whose endpoints intersect with at least two parcellation regions after all tissue/waymask/minimum-length filtering. This keeps runtimes down to <60 minutes for the complete structural connectometry pipeline. Bear in mind, however, that across our ensemble sample, we are **actually** sampling 20,000 x 72 = 1.4 million streamlines! And since we are further employing ensemble tractography, which samples across 5 step sizes (0.1, 0.2, 0.3, 0.4, 0.5) and 2 curvature thresholds (30, 40) by default, we are actually indirectly sampling across a much wider grid of parameters still.

Whereas in the functional connectometry case, we examined the mean connectome across estimates, here we might choose to examine the max connectome specifically, since structural connectomes are inherently sparser.

In [ ]:
%matplotlib inline
import pickle
import glob
import numpy as np
from matplotlib import pyplot as plt
from nilearn.plotting import plot_matrix
from pynets.core.thresholding import standardize

mats = [np.load(i) for i in glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi/BrainnetomeAtlasFan2016/graphs/*.npy')]

max_mat = standardize(np.max(mats, axis=0))


plot_matrix(
    max_mat,
    figure=(10, 10),
    labels=[' ']*len(max_mat),
    vmax=np.percentile(max_mat[max_mat > 0], 95),
    vmin=np.percentile(max_mat[max_mat > 0], 10),
    reorder="average",
    auto_fit=True,
    grid=False,
    colorbar=False,
    cmap='RdBu_r',
)
plt.show()

As you can see, we get a much more information-rich graph. This graph, unlike the first, now represents a new *distribution* of connectomes, that, by virtue of its plurality of views, more exhaustively samples from the true *population* of networks in this individual that may exhibit connectivity, across the whole brain as a region of interest.

# Collecting Outputs
Now, we explored the outputs of our connectome ensemble visually, but let's take a closer look at the actual topological data. To do this, we run another workflow using the `pynets_collect` CLI.

In [ ]:
%%bash

dir=~/Downloads/.pynets
abs_dir=`echo "$(dirname $dir)"`
pynets_collect -basedir "$abs_dir"/.pynets/outputs -modality 'dwi'

And as before we can view the output data and visualize resulting embeddings...

In [ ]:
import pathlib
from pathlib import Path
import pandas as pd

# Now we can load a dataframe containing all the AUC topological graph metrics calculated for this particular subject's run:
p = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/dwi_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'
df_individual = pd.read_csv(p, index_col=False)
df_individual

# Note that if we were to sample connectomes from multiple subjects, the previous pynets_collect CLI would
# simply append new rows to the summary `all_subs_neat_func.csv` dataframe.
p = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/all_subs_neat_dwi.csv'
df_group = pd.read_csv(p, index_col=False)
df_group

In [ ]:
%matplotlib inline
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
from pynets.plotting.plot_gen import plot_graph_measure_hists

csv_all_metrics = str(Path('~').expanduser()) + '/Downloads/.pynets/outputs/dwi_group_topology_auc/sub-OAS31172_ses-d0407_topology_auc_clean.csv'

out = plot_graph_measure_hists(csv_all_metrics)
out.show()

In [ ]:
%matplotlib inline
from graspy.plot import pairplot
import glob
import os
import numpy as np
from matplotlib import pyplot as plt

omnetomes = [[np.load(i), os.path.basename(i).split('.npy')[0]] for i in glob.glob('/Users/*/Downloads/.pynets/outputs/sub-OAS31172/ses-d0407/dwi/*/embeddings/*omnetome.npy')]

for grad, title in omnetomes:
    plot = pairplot(grad, title=title + '_Structural')

Later tutorials will cover a variety of additional topics, including how you can deploy PyNets across entire BIDS datasets in a single command-line interface (CLI) call, benchmark and optimize connectome ensembles across diverse analytic scenarios with GridSearchCV integration, along with more advanced topics such as performing and visualizing multiplex graph analysis and embeddings of multimodal connectomes.